In [368]:
import pandas as pd

In [369]:
training_data = pd.read_csv('train.csv')

In [370]:
test_data = pd.read_csv('test.csv')

In [371]:
X_train = training_data.copy()
#X_prepared = full_pipeline.fit_transform(X_train)
#X_train_prepared.shape
#X_test_prepared = full_pipeline.transform(test_data)
#X_test_prepared.shape

In [372]:
y_train = training_data["electricity_consumption"].copy()

In [373]:
from sklearn.preprocessing import LabelBinarizer

In [374]:
class LabelBinarizer_new(TransformerMixin, BaseEstimator):
    def fit(self, X, y = 0):
        self.encoder = None
        return self
    def transform(self, X, y = 0):
        if(self.encoder is None):
            print("Initializing encoder")
            self.encoder = LabelBinarizer();
            result = self.encoder.fit_transform(X)
        else:
            result = self.encoder.transform(X)
        return result; 

In [375]:
from sklearn.base import TransformerMixin,BaseEstimator

In [376]:
import numpy as np
windspeed_ix, pressure_ix, temperature_ix, var1_ix = 3, 2, 0, 1
class CombinedAttributesAdder(BaseEstimator,TransformerMixin):
    def __init__(self,add_features = True):
        self.add_features = True
    def fit(self,X,y=None):
        return self
    def transform(self,X ,y=None):
        windspeed_by_pressure = X[:,windspeed_ix]/X[:,pressure_ix]
        windspeed_into_pressure = X[:,windspeed_ix]*X[:,pressure_ix]
        temperature_by_pressure = X[:,temperature_ix]/X[:,pressure_ix]
        temperature_into_pressure = X[:,temperature_ix]*X[:,pressure_ix]
        pressure_into_var1 = X[:,pressure_ix]*X[:,var1_ix]
        windspeed_into_var1 = X[:,windspeed_ix]*X[:,var1_ix]
        return np.c_[X,windspeed_by_pressure,windspeed_into_pressure,temperature_by_pressure,temperature_into_pressure,pressure_into_var1,windspeed_into_var1]

In [377]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [378]:
num_attribs = ["temperature","var1","pressure","windspeed"]
cat_attribs = ["var2"]
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
num_pipeline = Pipeline([('selector',DataFrameSelector(num_attribs)),('attribs_adder',CombinedAttributesAdder()),('std_scaler',StandardScaler())])
cat_pipeline = Pipeline([('selector',DataFrameSelector(cat_attribs)),('label_binarizer',LabelBinarizer_new())])

In [379]:
from sklearn.pipeline import FeatureUnion

In [380]:
full_pipeline = FeatureUnion(transformer_list=[("num_pipeline",num_pipeline),("cat_pipeline",cat_pipeline)])

In [381]:
X_prepared = full_pipeline.fit_transform(X_train)

Initializing encoder


In [382]:
X_prepared.shape

(26496, 13)

In [383]:
days = np.array([8*24,8*24,7*24,8*24,7*24,8*24,8*24,5*24,8*24,7*24,8*24,7*24,8*24,8*24,7*24,8*24,7*24,8*24,8*24,5*24,8*24,7*24,8*24,7*24,8*24,8*24,7*24,8*24,7*24,8*24,8*24,6*24,8*24,7*24,8*24,7*24,8*24,8*24,7*24,8*24,7*24,8*24,8*24,5*24,8*24,7*24,8*24,7*24])
days = np.cumsum(days)
from sklearn.svm import SVR
svr_reg = SVR(kernel = "rbf",gamma = 5,C=0.001)

In [385]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(n_estimators=100)
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
from operator import add
i=0
k=[]
p=[]
r=[]
for n_estimators in range(1,49):
    #forest_reg.n_estimators =  100*n_estimators
    #forest_reg.fit(X_prepared[:n_estimators*24*23],y_train[:n_estimators*24*23])
    #lin_reg.fit(X_prepared[(n_estimators-1)*24*23:n_estimators*24*23],y_train[(n_estimators-1)*24*23:n_estimators*24*23])
    forest_reg.fit(X_prepared[(n_estimators-1)*24*23:n_estimators*24*23],y_train[(n_estimators-1)*24*23:n_estimators*24*23])
    #p.append(lin_reg)
    r.append(forest_reg)
    print(n_estimators)
    X_test = test_data[i:days[n_estimators-1]].copy()
    i=days[n_estimators-1]
    y_id = X_test['ID'].values
    X_test_prepared = full_pipeline.transform(X_test)
    y_pred = [0]
    y_pred = y_pred*X_test_prepared.shape[0]
    for lin_reg in r:
        y_pred =map(add,y_pred, lin_reg.predict(X_test_prepared))
    div=len(r)
    y_pred = [x/div for x in y_pred]
    y_pred = np.c_[y_id,y_pred]
    if n_estimators==1:
        k = y_pred
    else:
        k = np.r_[k,y_pred]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [386]:
k

array([[  552.  ,   205.5 ],
       [  553.  ,   205.68],
       [  554.  ,   203.7 ],
       ...,
       [35061.  ,   200.67],
       [35062.  ,   194.7 ],
       [35063.  ,   197.94]])

In [387]:
k = pd.DataFrame(k,columns = ["ID","electricity_consumption"])

In [388]:
k.head()

,ID,electricity_consumption
0,552.0,205.50
1,553.0,205.68
2,554.0,203.70
3,555.0,202.56
4,556.0,253.65


In [389]:
we = k.to_csv('submission.csv',index=False)